**LoRA implementation with Pytorch**

In [44]:
import torch
import torchvision.datasets as datasets 
import torchvision.transforms as transforms
import torch.nn as nn
import matplotlib.pyplot as plt
from tqdm import tqdm

_ = torch.manual_seed(0)

In [45]:
# Image preproccess pipeline
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])

# Load the MNIST dataset
mnist_trainset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
# Create a dataloader for the training
train_loader = torch.utils.data.DataLoader(mnist_trainset, batch_size=10, shuffle=True)

# Load the MNIST test set
mnist_testset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(mnist_testset, batch_size=10, shuffle=True)

# Define the device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


Create the Neural Network to classify the digits, making it overly complicated to better show the power of LoRA

In [46]:
# Create an overly expensive neural network to classify MNIST digits
class RichNet(nn.Module):
    def __init__(self, hidden_size_1 = 1000, hidden_size_2=2000):
        super(RichNet,self).__init__()
        self.linear1 = nn.Linear(28*28, hidden_size_1) 
        self.linear2 = nn.Linear(hidden_size_1, hidden_size_2) 
        self.linear3 = nn.Linear(hidden_size_2, 10)
        self.relu = nn.ReLU()

    def forward(self, img):
        x = img.view(-1, 28*28) # [Batch_size, 784]
        x = self.relu(self.linear1(x))
        x = self.relu(self.linear2(x))
        x = self.linear3(x)
        return x
    
net = RichNet().to(device)
print(net.linear1.weight.shape)

torch.Size([1000, 784])


Train the network only for 1 epoch to simulate a complete general pre-training on the data

In [47]:
def train(train_loader, net, epochs=5, total_iterations_limit=None):
    cross_el = nn.CrossEntropyLoss() #Input: unnormalized logits; output: [Batch_size, Num_class]; y: [Batch_si、
    optimizer = torch.optim.Adam(net.parameters(), lr=0.001)

    total_iterations = 0

    for epoch in range(epochs):
        net.train()

        loss_sum = 0
        num_iterations = 0

        data_iterator = tqdm(train_loader, desc=f'Epoch {epoch+1}')
        if total_iterations_limit is not None:
            data_iterator.total = total_iterations_limit
        for data in data_iterator:   # iteration_num = len(dataset) / batch_size
            num_iterations += 1
            total_iterations += 1
            x, y = data # x: [Batch_size, Num_channel, H, W]; # y: [Batch_size]
            x = x.to(device) 
            y = y.to(device)
            optimizer.zero_grad()
            output = net(x.view(-1, 28*28))
            loss = cross_el(output, y) # A scalar, average loss over the batch
            loss_sum += loss.item()
            avg_loss = loss_sum / num_iterations
            data_iterator.set_postfix(loss=avg_loss)
            loss.backward()
            optimizer.step()

            if total_iterations_limit is not None and total_iterations >= total_iterations_limit:
                return

train(train_loader, net, epochs=1)

Epoch 1: 100%|██████████| 6000/6000 [00:16<00:00, 356.03it/s, loss=0.239]


Keep a copy of the original weights (cloning them) so later we can prove that a fine-tuning with LoRA doesn't alter the original weights

In [48]:
original_weights = {}
for name, param in net.named_parameters():
    original_weights[name] = param.clone().detach()
# Make a clean numeric snapshot of each parameter:
# clone() creates a new tensor with its own memory (not a reference),
# detach() removes it from the computation graph (no gradients tracked).
# This ensures that even if the model parameters are updated later,
# the values stored in original_weights remain unchanged.

In [49]:
def test():
    correct = 0
    total = 0

    wrong_counts = [0 for i in range(10)]

    with torch.no_grad():
        for data in tqdm(test_loader, desc='Testing'):
            x, y = data
            x = x.to(device)
            y = y.to(device)
            output = net(x.view(-1, 784))
            preds = torch.argmax(output, dim=1)  # [batch_size]
            correct_mask = (preds == y)
            correct = correct + correct_mask.sum().item()
            total += y.size(0)
            for label in range(10):
                wrong_counts[label] += ((y == label) & (preds != label)).sum().item()
    print(f'Accuracy: {round(correct/total, 3)}')
    for i in range(len(wrong_counts)):
        print(f'wrong counts for the digit {i}: {wrong_counts[i]}')

test()

Testing: 100%|██████████| 1000/1000 [00:02<00:00, 489.53it/s]

Accuracy: 0.957
wrong counts for the digit 0: 14
wrong counts for the digit 1: 21
wrong counts for the digit 2: 65
wrong counts for the digit 3: 47
wrong counts for the digit 4: 37
wrong counts for the digit 5: 55
wrong counts for the digit 6: 23
wrong counts for the digit 7: 53
wrong counts for the digit 8: 10
wrong counts for the digit 9: 102


As we can see, the original richnet performs worst w.r.t digit 7, let's definr a lora fintuning on digit 9.

In [50]:
total_original_parameters = 0
for index, layer in enumerate([net.linear1, net.linear2, net.linear3]):
    total_original_parameters += (layer.weight.nelement() + layer.bias.nelement())
    print(f"Layer {index+1}: shape of weight: {layer.weight.shape}; shape of bias: {layer.bias.shape}")
print(f"Total number of parameters: {total_original_parameters}")

Layer 1: shape of weight: torch.Size([1000, 784]); shape of bias: torch.Size([1000])
Layer 2: shape of weight: torch.Size([2000, 1000]); shape of bias: torch.Size([2000])
Layer 3: shape of weight: torch.Size([10, 2000]); shape of bias: torch.Size([10])
Total number of parameters: 2807010


In [51]:
class LoRAParameterization(nn.Module):
    def __init__(self, features_in, features_out, rank=1, alpha=1, device='cuda'):
        super().__init__()
        self.lora_A = nn.Parameter(torch.zeros((rank, features_in)).to(device))
        self.lora_B = nn.Parameter(torch.zeros((features_out, rank)).to(device))
        nn.init.normal_(self.lora_A, mean=0, std=1)

        self.scale = alpha/rank
        self.enabled = True
    
    def forward(self, original_weights):
        if self.enabled:
            return original_weights + torch.matmul(self.lora_B, self.lora_A).view(original_weights.shape) * self.scale
        else:
            return original_weights

Add the Parametrization to our network.

In [52]:
import torch.nn.utils.parametrize as parametrize

def linear_layer_parameterization(layer, device, rank=1, lora_alpha=1):
    features_out, features_in = layer.weight.shape
    return LoRAParameterization(
        features_in, features_out, rank=rank, alpha=lora_alpha, device=device
    )

parametrize.register_parametrization(
    net.linear1, "weight", linear_layer_parameterization(net.linear1, device)
)
parametrize.register_parametrization(
    net.linear2, "weight", linear_layer_parameterization(net.linear2, device)
)
parametrize.register_parametrization(
    net.linear3, "weight", linear_layer_parameterization(net.linear3, device)
)

def enable_disable_lora(enabled=True):
    for layer in [net.linear1, net.linear2, net.linear3]:
        layer.parametrizations["weight"][0].enabled = enabled



Display the number of parameters added by LoRA.

In [53]:
total_parameters_lora = 0
total_parameters_non_lora = 0
for index, layer in enumerate([net.linear1, net.linear2, net.linear3]):
    total_parameters_lora += layer.parametrizations["weight"][0].lora_A.nelement() + layer.parametrizations["weight"][0].lora_B.nelement()
    total_parameters_non_lora += layer.weight.nelement() + layer.bias.nelement()
    print(
        f'Layer {index+1}: W: {layer.weight.shape} + B: {layer.bias.shape} + Lora_A: {layer.parametrizations["weight"][0].lora_A.shape} + Lora_B: {layer.parametrizations["weight"][0].lora_B.shape}'
    )
# The non-LoRA parameters count must match the original network
assert total_parameters_non_lora == total_original_parameters
print(f'Total number of parameters (original): {total_parameters_non_lora:,}')
print(f'Total number of parameters (original + LoRA): {total_parameters_lora + total_parameters_non_lora:,}')
print(f'Parameters introduced by LoRA: {total_parameters_lora:,}')
parameters_incremment = (total_parameters_lora / total_parameters_non_lora) * 100
print(f'Parameters incremment: {parameters_incremment:.3f}%')

Layer 1: W: torch.Size([1000, 784]) + B: torch.Size([1000]) + Lora_A: torch.Size([1, 784]) + Lora_B: torch.Size([1000, 1])
Layer 2: W: torch.Size([2000, 1000]) + B: torch.Size([2000]) + Lora_A: torch.Size([1, 1000]) + Lora_B: torch.Size([2000, 1])
Layer 3: W: torch.Size([10, 2000]) + B: torch.Size([10]) + Lora_A: torch.Size([1, 2000]) + Lora_B: torch.Size([10, 1])
Total number of parameters (original): 2,807,010
Total number of parameters (original + LoRA): 2,813,804
Parameters introduced by LoRA: 6,794
Parameters incremment: 0.242%


Freeze all the parameters of the original network and only fine tuning the ones introduced by LoRA. Then fine-tune the model on the digit 9 and only for 100 batches.

In [54]:
# Freeze the non-lora parameters
for name, param in net.named_parameters():
    if "lora" not in name:
        print(f'Freezing non-LoRA parameter {name}')
        param.requires_grad = False

# Load the MNIST dataset again, by keeping only the digit 9
mnist_trainset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
exclude_indices = mnist_trainset.targets == 9
mnist_trainset.data = mnist_trainset.data[exclude_indices]
mnist_trainset.targets = mnist_trainset.targets[exclude_indices]
# Create a dataloader for the training
train_loader = torch.utils.data.DataLoader(mnist_trainset, batch_size=10, shuffle=True)

# Train the network with LoRA only on the digit 9 
train(train_loader, net, epochs=1, total_iterations_limit=100)

Freezing non-LoRA parameter linear1.bias
Freezing non-LoRA parameter linear1.parametrizations.weight.original
Freezing non-LoRA parameter linear2.bias
Freezing non-LoRA parameter linear2.parametrizations.weight.original
Freezing non-LoRA parameter linear3.bias
Freezing non-LoRA parameter linear3.parametrizations.weight.original


Epoch 1:   0%|          | 0/100 [00:00<?, ?it/s, loss=0.135]

Epoch 1:  99%|█████████▉| 99/100 [00:00<00:00, 300.63it/s, loss=0.109]


Verify that the fine-tuning didn't alter the original weights, but only the ones introduced by LoRA.

In [55]:
# Check that the frozen parameters are still unchanged by the finetuning
assert torch.all(net.linear1.parametrizations.weight.original == original_weights['linear1.weight'])
assert torch.all(net.linear2.parametrizations.weight.original == original_weights['linear2.weight'])
assert torch.all(net.linear3.parametrizations.weight.original == original_weights['linear3.weight'])

enable_disable_lora(enabled=True)
# The new linear1.weight is obtained by the "forward" function of our LoRA parametrization
# The original weights have been moved to net.linear1.parametrizations.weight.original
# More info here: https://pytorch.org/tutorials/intermediate/parametrizations.html#inspecting-a-parametrized-module
assert torch.equal(net.linear1.weight, net.linear1.parametrizations.weight.original + (net.linear1.parametrizations.weight[0].lora_B @ net.linear1.parametrizations.weight[0].lora_A) * net.linear1.parametrizations.weight[0].scale)

enable_disable_lora(enabled=False)
# If we disable LoRA, the linear1.weight is the original one
assert torch.equal(net.linear1.weight, original_weights['linear1.weight'])

Test the network with LoRA enabled (the digit 9 should be classified better)

In [56]:
# Test with LoRA enabled
enable_disable_lora(enabled=True)
test()

enable_disable_lora(enabled=False)
test()

Testing: 100%|██████████| 1000/1000 [00:02<00:00, 445.20it/s]


Accuracy: 0.897
wrong counts for the digit 0: 19
wrong counts for the digit 1: 25
wrong counts for the digit 2: 84
wrong counts for the digit 3: 104
wrong counts for the digit 4: 187
wrong counts for the digit 5: 150
wrong counts for the digit 6: 23
wrong counts for the digit 7: 286
wrong counts for the digit 8: 140
wrong counts for the digit 9: 11


Testing: 100%|██████████| 1000/1000 [00:02<00:00, 485.61it/s]

Accuracy: 0.957
wrong counts for the digit 0: 14
wrong counts for the digit 1: 21
wrong counts for the digit 2: 65
wrong counts for the digit 3: 47
wrong counts for the digit 4: 37
wrong counts for the digit 5: 55
wrong counts for the digit 6: 23
wrong counts for the digit 7: 53
wrong counts for the digit 8: 10
wrong counts for the digit 9: 102
